In [1]:
def unify_terms(t1, t2):
    """
    Attempts to unify two terms and returns the substitution if successful, otherwise returns "FAILURE".
    """
    # If the terms are the same (constant or identical terms)
    if t1 == t2:
        return {}

    # If t1 is a variable, substitute it with t2
    if is_variable(t1):
        return substitute(t1, t2) if not variable_occurs_in(t1, t2) else "FAILURE"

    # If t2 is a variable, substitute it with t1
    if is_variable(t2):
        return substitute(t2, t1) if not variable_occurs_in(t2, t1) else "FAILURE"

    # If the predicates don't match, unification fails
    if get_predicate(t1) != get_predicate(t2):
        return "FAILURE"

    # If the arguments' lengths don't match, unification fails
    if len(get_arguments(t1)) != len(get_arguments(t2)):
        return "FAILURE"

    # Perform unification for each argument pair
    substitution = {}

    for arg1, arg2 in zip(get_arguments(t1), get_arguments(t2)):
        sub = unify_terms(apply_substitution(arg1, substitution), apply_substitution(arg2, substitution))
        if sub == "FAILURE":
            return "FAILURE"
        substitution.update(sub)  # Merge substitutions

    return substitution


def is_variable(term):
    """Checks if the term is a variable (lowercase string)."""
    return isinstance(term, str) and term.islower()


def variable_occurs_in(var, term):
    """
    Checks if a variable occurs within a term (or in any subterms if the term is a compound term).
    """
    if var == term:
        return True
    elif isinstance(term, (list, tuple)):
        return any(variable_occurs_in(var, t) for t in term)
    return False


def get_predicate(term):
    """Returns the predicate symbol of a term."""
    return term[0] if isinstance(term, list) else None


def get_arguments(term):
    """Returns the arguments of a term (ignoring the predicate symbol)."""
    return term[1:] if isinstance(term, list) else []


def substitute(var, value):
    """Substitutes a variable with a value (only if the variable is not already in the value)."""
    return {var: value} if var != value else "FAILURE"


def apply_substitution(term, substitution):
    """
    Applies the substitution to a term.
    If the term is a variable, it checks the substitution dictionary.
    If it's a complex term (like a list), it applies the substitution recursively.
    """
    if is_variable(term):
        return substitution.get(term, term)
    elif isinstance(term, list):
        return [apply_substitution(t, substitution) for t in term]
    return term


def term_to_string(term):
    """Converts a term into a human-readable string."""
    if isinstance(term, list):
        predicate_name = term[0]
        arguments_str = ", ".join(term_to_string(arg) for arg in term[1:])
        return f"{predicate_name}({arguments_str})"
    return str(term)


# Test case
if __name__ == "__main__":
    t1 = ['Eats', 'x', 'Apple']
    t2 = ['Eats', 'Riya', 'y']

    result = unify_terms(t1, t2)

    if result != "FAILURE":
        unified_t1 = apply_substitution(t1, result)
        print(term_to_string(unified_t1))  # Expected: Eats(Riya, Apple)
    else:
        print("Unification Failed")

Eats(Riya, Apple)
